In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

from ak_diseq.glacierData import glacierData
from ak_diseq.utils import weighted_quantile, hist2

plt.style.use('default')


In [ ]:
# Zeller dataset
elas = pd.read_csv(Path('data/zeller/alaska_elas.csv'))
print(elas.head(5))
elas.aar.describe()

In [ ]:
# 0.4, 0.6, and Zeller 2024 AAR.
gd_aar40 = glacierData(aar=0.4)
gd_aar60 = glacierData(aar=0.6)
gd_aarZ = glacierData(aar="zeller")
gds = {
    "0.4": gd_aar40,
    "0.6": gd_aar60,
    'zeller': gd_aarZ
}
ds = {}
for key, gd in gds.items():
    gd.calc_response_time()
    gd.calc_linear_feq()
    gd.calc_feq()
    ds[key] = gd.rgi

In [ ]:
for aar, d in ds.items():
    print(aar, len(d))

In [ ]:
#d.apply(lambda x: x['Zela'] - x['z'][x['Zela_idx']], axis=1)
fig, ax = plt.subplots(1,1)
for aar, d in ds.items():
    sns.histplot(d['bt'], label=aar, fill=False, ax=ax, bins=np.linspace(-20, 1, 40), element='step')
    print(f"\n{aar}")
    print("Median:", weighted_quantile(d["bt"], [0.05, 0.5, 0.95])) 
    print("weighted median:", weighted_quantile(d["bt"], [0.05, 0.5, 0.95], sample_weight=d["Area"]))
    print("Std:", d['bt'].std())
    print("Skew:", d['bt'].skew())
ax.legend()



In [ ]:
fig, ax = plt.subplots(2,1)
binfreq = 0.25
for aar, d in ds.items():
    x = d['bt']
    uwpdf, bins = hist2(x, binfreq=binfreq, cumsum=False, from_zero=True)
    awcdf, bins = hist2(x, area=d['Area'].to_numpy(), binfreq=binfreq, cumsum=True, from_zero=True)
    ax[0].step(bins, uwpdf, label=aar)
    ax[1].step(bins, awcdf)
    ax[0].legend()
    ax[1].set_xlabel("bt [m yr-1]")
    
    print(f"\n{aar}")
    print("Median:", weighted_quantile(x, [0.05, 0.5, 0.95])) 
    print("weighted median:", weighted_quantile(x, [0.05, 0.5, 0.95], sample_weight=d["Area"]))
    print("Std:", x.std())
    print("Skew:", x.skew())

In [ ]:
ds_plt = {key: df.loc[:, ["Area", "bt", "Zela", "RGIId"]] for key, df in ds.items()}
d = ds_plt["0.4"]
qcuts = pd.cut(d['Area'], bins=[1,2, 5, 10, 25, 50, 100, 250, 1000, 10000], precision=0)

for key in ["0.6", "zeller"]:
    d_merge = ds_plt[key][[col for col in ds_plt[key] if col not in ['Area']]]  # get the df from the dict of cases
    print(key)
    print(d.columns)
    print(d_merge.columns)
    d = d.merge(d_merge, on=["RGIId"], suffixes=["_0.4", f"_{key}"])  #
    print(d.columns)
d = d.rename(columns={"bt": "bt_zeller", "Zela": "Zela_zeller", "aar": "aar_zeller"})
print(d.columns)
d["qcut"] = qcuts
# Reshape the DataFrame using melt for Seaborn plotting
d = pd.melt(
    d,
    id_vars=["RGIId", "Area", "qcut"],
    value_vars=[col for col in d.columns if col not in ["RGIId", "Area" "qcut"]],
    var_name="variable",
    value_name="value"
)

print(d.columns)
# Extract suffixes and assign them as a new column for hue
d["suffix"] = d["variable"].apply(lambda x: x.split("_")[-1]).astype(str)
d["variable"] = d["variable"].apply(lambda x: x.split("_")[0])


print(d.columns)

fig, ax = plt.subplots(layout='constrained', dpi=200)
sns.boxplot(data=d.loc[d.variable == "bt"], x="qcut", y="value", hue='suffix', ax=ax, hue_order=["zeller", "0.4", "0.6"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
(
    so.Plot(x=qcuts.cat.categories, y=0, text=[f"{item:.0f}" for item in qcuts.value_counts().values/2])
    .add(so.Text(fontsize='xx-small',valign='top', offset=5))
    .on(ax)
    .plot()
)
ax.grid(which='both', axis='y')
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(4))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(1))
ax.set_ylim([-20, 1])
ax.set_xlabel('Area (km2)')
ax.set_ylabel('bt (m swe yr^-1)')

In [ ]:

Zela_mean = d.loc[d.variable == "Zela"].groupby("suffix").mean(numeric_only=True)
Zela_mean = Zela_mean["value"]

print("Mean Zela:", Zela_mean)

diff = (Zela_mean["0.4"] - Zela_mean["0.6"])
print("Pct. diff (0.4 - 0.6): ", diff/Zela_mean["0.6"]) 
print("Diff (0.4 - 0.6)",diff, "\n")

diff = (Zela_mean["zeller"] - Zela_mean["0.4"])
print("Pct. diff (Zeller - 0.4): ", diff/Zela_mean["0.4"]) 
print("Diff (Zeller - 0.4): ", diff)

In [ ]:
# Area-weighted ela
dd = d.loc[d.variable == "Zela"].groupby("suffix").apply(lambda x: weighted_quantile(x["value"], [0.05, 0.5, 0.95]), include_groups=False)

dd_aw = d.loc[d.variable == "Zela"].groupby("suffix").apply(lambda x: weighted_quantile(x["value"], [0.05, 0.5, 0.95], sample_weight=x["Area"]), include_groups=False)

print("Number weighted:")
print(dd, "\n")

print("Area weighted:")
print(dd_aw)

In [ ]:
dd = d.loc[d.variable == "Zela"]
print("Median Zela:", weighted_quantile(dd["value"], [0.05, 0.5, 0.95]))
print("Weighted median Zela:", weighted_quantile(dd["value"], [0.05, 0.5, 0.95], sample_weight=dd["Area"]))

In [ ]:
# what is the pct difference in bt for each glacier?
dd = d.loc[d.variable == "bt"]
dd = dd.pivot_table(columns=['suffix'], index=["RGIId", "Area"], values='value').reset_index()
diff = dd["0.4"]/dd["0.6"] - 1
print(diff.describe())
print("Weighted medianx pct diff:", weighted_quantile(diff, [0.05, 0.5, 0.95], sample_weight=dd["Area"]))

diff = dd["zeller"]/dd["0.4"] - 1
print(diff.describe())
print("Weighted median pct diff:", weighted_quantile(diff, [0.05, 0.5, 0.95], sample_weight=dd["Area"]))

In [ ]:
#d.apply(lambda x: x['Zela'] - x['z'][x['Zela_idx']], axis=1)
fig, ax = plt.subplots(1,1)
for aar, d in ds.items():
    sns.histplot(d['tau'], label=aar, fill=False, ax=ax, bins=np.linspace(0, 100, 40), element='step')
ax.legend()

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True)
binfreq = 2.5
for aar, d in ds.items():
    x = d['tau']
    uwpdf, bins = hist2(x, binfreq=binfreq, cumsum=False, from_zero=True)
    awcdf, bins = hist2(x, area=d['Area'].to_numpy(), binfreq=binfreq, cumsum=True, from_zero=True)
    ax[0].step(bins, uwpdf, label=aar)
    ax[1].step(bins, awcdf)
    ax[0].legend()
    ax[0].set_xlim((0, 200))
    ax[0].set_xlabel("Tau [yr]")
    print(f"\n{aar}")
    print("Median:", weighted_quantile(x, [0.05, 0.5, 0.95])) 
    print("weighted median:", weighted_quantile(x, [0.05, 0.5, 0.95], sample_weight=d["Area"]))
    print("Std:", x.std())
    print("Skew:", x.skew())
    

In [ ]:
ds_plt = {key: df.loc[:, ["Area", "tau", "RGIId"]] for key, df in ds.items()}
d = ds_plt["0.4"]#.rename(columns={"tau": "tau_0.4"})
qcuts = pd.cut(d['Area'], bins=[1,2, 5, 10, 25, 50, 100, 250, 1000, 10000], precision=0)

for key in ["0.6", "zeller"]:
    d_merge = ds_plt[key][[col for col in ds_plt[key] if col not in ['Area']]]  # get the df from the dict of cases
    print(key)
    print(d.columns)
    print(d_merge.columns)
    d = d.merge(d_merge, on=["RGIId"], suffixes=["_0.4", f"_{key}"])  #
    print(d.columns)
d = d.rename(columns={"tau": "tau_zeller"})
print(d.columns)
d["qcut"] = qcuts
# Reshape the DataFrame using melt for Seaborn plotting
d = pd.melt(
    d,
    id_vars=["RGIId", "Area", "qcut"],
    value_vars=[col for col in d.columns if col not in ["RGIId", "Area" "qcut"]],
    var_name="variable",
    value_name="value"
)

# Extract suffixes and assign them as a new column for hue
d["suffix"] = d["variable"].apply(lambda x: x.split("_")[-1]).astype(str)
d["variable"] = d["variable"].apply(lambda x: x.split("_")[0])


print(d.columns)

In [ ]:
fig, ax = plt.subplots(layout='constrained', dpi=200)
sns.boxplot(data=d.loc[d.variable == "tau"], x="qcut", y="value", hue='suffix', ax=ax, hue_order=["zeller", "0.4", "0.6"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
(
    so.Plot(x=qcuts.cat.categories, y=0, text=[f"{item:.0f}" for item in qcuts.value_counts().values/2])
    .add(so.Text(fontsize='xx-small',valign='top', offset=5))
    .on(ax)
    .plot()
)
ax.grid(which='both', axis='y')
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(25))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(5))
ax.set_ylim([-10, 200])
ax.set_xlabel('Area (km2)')
ax.set_ylabel('Tau (yr)')
ax.set_title('Tau')

In [ ]:

fig, ax = plt.subplots(layout='constrained', dpi=200)
sns.boxplot(data=d.loc[d.variable == "tau"], x="qcut", y="value", hue='suffix', ax=ax, hue_order=["zeller", "0.4", "0.6"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
(
    so.Plot(x=qcuts.cat.categories, y=0, text=[f"{item:.0f}" for item in qcuts.value_counts().values/2])
    .add(so.Text(fontsize='xx-small',valign='top', offset=5))
    .on(ax)
    .plot()
)
ax.grid(which='both', axis='y')
ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(25))
ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(5))
ax.set_ylim([-10, 200])
ax.set_xlabel('Area (km2)')
ax.set_ylabel('pct diff')
ax.set_title('Tau')

In [ ]:
fig, ax = plt.subplots(1,1)
for aar, d in ds.items():
    sns.histplot(d['feq'], label=aar, fill=False, ax=ax, bins=np.linspace(0, 1, 40), element='step')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.hist(elas.aar, bins=30, label='aar', alpha=0.5)
ax.hist(elas.hyps_aar, bins=30, label='hyps_aar', alpha=0.5)
ax.legend()

In [ ]:
print(np.nanmean(elas.aar))
print(np.median(elas.aar))

In [ ]:
fig, ax = plt.subplots(2,1, sharex=True)
binfreq = 0.025
for aar, d in ds.items():
    x = d['feq_gwi']
    uwpdf, bins = hist2(x, binfreq=binfreq, cumsum=False, from_zero=True)
    awcdf, bins = hist2(x, area=d['Area'].to_numpy(), binfreq=binfreq, cumsum=True, from_zero=True)
    ax[0].step(bins, uwpdf, label=aar)
    ax[1].step(bins, awcdf)
    ax[0].legend()
    ax[0].set_xlim((0, 1))
    ax[0].set_ylabel("feq")
    print(f"\n{aar}")
    print("Median:", weighted_quantile(x, [0.05, 0.5, 0.95])) 
    print("weighted median:", weighted_quantile(x, [0.05, 0.5, 0.95], sample_weight=d["Area"]))
    print("Std:", x.std())
    print("Skew:", x.skew())
    

In [ ]:
cols = ['RGIId', 'hf', 'bt', 'tau', 'feq', 'feq_linear', 'feq_gwi', 'feq_etcw', 'Name', 'Zmin', 'Zmax', 'Zela']
ds_plt = {key: df.loc[:, cols] for key, df in ds.items()}

id_cols = ['RGIId', 'Name', 'Area', 'Zmin', 'Zmax', 'Zela']
d = None
for key in ["0.4", "0.6", "zeller"]:
    if d is None:
        # Initialize 'd' with the first dataframe and rename columns
        d = ds_plt[key]
        d.columns = [
            f"{col}_{key}" if col not in id_cols else col
            for col in d.columns
        ]
    else:
        # Select the dataframe to merge and exclude id_cols from renaming
        d_merge = ds_plt[key]
        
        # Select only the columns that are not in id_cols
        cols_to_merge = [col for col in d_merge.columns if col not in id_cols]
        
        # Rename columns to include the key suffix
        d_merge = d_merge[cols_to_merge]
        d_merge.columns = [
            f"{col}_{key}" for col in d_merge.columns
        ]
        
        print(f"Merging key: {key}")
        print(f"d columns: {d.columns}")
        print(f"d_merge columns: {d_merge.columns}")
        
        # Merge dataframes on id_cols
        d = d.merge(d_merge, left_index=True, right_index=True)  # Merge using index as keys match through index
        print(f"After merge: {d.columns}")



In [ ]:
combined_df = pd.concat(ds, names=['case', 'index'])
# Set the multi-index to include 'RGIId' and 'case'
combined_df = combined_df.set_index(['RGIId'], append=True).swaplevel('RGIId', 'index').sort_index().reset_index(level='index', drop=True)

cols_to_keep = ['Name', 'Area', 'bt', 'hf', 'tau', 'feq', 'feq_linear', 'feq_gwi', 'feq_etcw', 'Zmin', 'Zmax', 'Zela', 'Lmax', 'aar', 'hyps_ela', 'hyps_aar']
combined_df = combined_df[cols_to_keep]
combined_df.head(10)

In [ ]:
p = Path("TableD1_data.csv")
combined_df.to_csv(p)